In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# endregion       #
# --------------- #

In [2]:
%autoreload 2
from gem.models.transformer import *
import torch

state = torch.zeros(64, 100, 3, 15, 15, requires_grad=True)
action = torch.randint(0, 3, (64, 100, 1))

vit = VisionTransformer(
    state_size=np.array(state.size()[2:]),
    action_space=5,
    layer_size=100,
    patch_size=3,
    num_frames=int(state.size()[1]),
    batch_size=int(state.size()[0]),
    num_layers=4,
    num_heads=4,
    memory=ReplayBuffer(0,0,'cpu',0,0,5),
    LR = .001,
    device='cpu',
    seed=1
)

In [3]:
out = vit.forward(
    state, action
)

for o in out:
    print(o.size())

vit.action_loss(
    out[1], action.squeeze()
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6400x4100 and 2600x100)

In [4]:
# %autoreload 2
# from examples.ft.test import eval_model

# results = eval_model(
#     'memories',
#     model_path='../models/checkpoints/model_iRainbowModel_pomdp_0.0.pkl',
#     config_path='../config.yaml',
#     n_games=1000
# )

# results['memories'].save('../models/checkpoints/pomdp_memories.pkl')

In [5]:
%autoreload 2
import argparse
import numpy as np
from IPython.display import clear_output
from matplotlib import pyplot as plt

from gem.utils import fig2img, animate
from gem.models.inverse import MLPBCModel
from gem.models.buffer import ActionBatchReplayBuffer as ABRBuffer
from gem.config import load_config, Cfg

from examples.trucks.config import create_agents, create_entities, create_models
from examples.trucks.env import FoodTrucks

# ------------------------------------------------------ #
# region: Create the forward model for the animations... #
# ------------------------------------------------------ #

cfg = load_config(argparse.Namespace(config='../config.yaml'))
models = create_models(cfg)
agents = create_agents(cfg, models)
entities = create_entities(cfg)
env = FoodTrucks(cfg, agents, entities)

# ------------------------------------------------------ #
# endregion                                              #
# ------------------------------------------------------ #


memories = ABRBuffer.load('../models/checkpoints/pomdp_memories.pkl')
inverse_model = MLPBCModel(
    state_size=cfg.model.iqn.parameters.state_size,
    action_size=5,
    layer_size=5000,
    epsilon=0.,
    n_channels=7,
    frames=5,
    seed=np.random.randint(0,1000),
    device='cpu',
    memory=memories,
    LR=.00001
)

losses = []

action_model_store = ABRBuffer(
    buffer_size=cfg.experiment.max_turns,
    batch_size=cfg.experiment.max_turns,
    device=None,
    seed=None,
    gamma=None,
    timesteps=5,
    action_space=5
)

for i in range(100000):

    action_loss, reconstruction_loss = inverse_model.train_model()
    if i % 50 == 0:
        clear_output(wait = True)

        # ----------------- #
        # region: Game loop #
        # ----------------- #

        env.reset()
        for agent in agents:
            agent.reset()
            # Load model checkpoint
            agent.model.load('../models/checkpoints/model_iRainbowModel_pomdp_0.0.pkl')
            # Zero out model epsilon
            agent.model.set_epsilon(0)
            # Evaluation mode
            agent.model.eval()

        done, turn = 0, 0

        while done == 0:

            # ----------------- #
            # region: Turn loop #
            # ----------------- #

            for agent in agents:

                state, action, reward, next_state, done_ = agent.transition(env)

                # If agent finished or max turns is reached, end the turn
                if turn >= cfg.experiment.max_turns or done_: 
                    done = 1 # End loop if maximum turns have been reached
                
                # Append the experience to the replay
                exp = (0, (state, action, reward, next_state, done))
                agent.episode_memory.append(exp)
                action_model_store.add(state, action, reward, next_state, done)

            # ----------------- #
            # endregion         #
            # ----------------- #

        frames = []

        while len(action_model_store.memory) > 0:

            experience = action_model_store.memory.popleft()
            state, action, _, next_state, _ = experience

            clear_output(wait = True)
            fig = inverse_model.plot_images(state, action.squeeze(), next_state)
            img = fig2img(fig)
            plt.show()
            frames.append(img)

        animate(frames,
                f'inverse_model_{i}.png',
                '../data/pomdp/')
                    
        # ----------------- #
        # endregion         #
        # ----------------- #

ValueError: Config file not found, please make sure you've included a path to the config file.